### **2-1. 객체 및 관계 추출에 필요한 library & function import.**

In [ ]:
from executors.completion_executor import ChatCompletionExecutor
from executors.sliding_window_executor import SlidingWindowExecutor
import json

### **2-2. Sliding Window Executor와 Completion Executor 선언.**

- Sliding Window Executor : Hyperclova X와의 대화에서 대화 내용이 길어져서 Token의 한도가 넘어가는 경우, 기록을 삭제하여 오류를 방지.

- Completion Executor : Hyperclova X의 응답을 받는 executor. (model : HCX-003)

In [ ]:
sliding_window_executor = SlidingWindowExecutor(
    host='clovastudio.apigw.ntruss.com',
    api_key='NTA0MjU2MWZlZTcxNDJiY+Uy9rYumYZsV6izDa9AFvO56MDOrkyLi3eCmugWbNRu',
    api_key_primary_val = '1yiWJOyxczG1Ichq95JEUty5cZKr3F5iBynZfQWL',
    request_id='3dd95651-1c46-47e5-b56a-17b24d80669f'
)

completion_executor = ChatCompletionExecutor(
    host='https://clovastudio.stream.ntruss.com',
    api_key='NTA0MjU2MWZlZTcxNDJiY+Uy9rYumYZsV6izDa9AFvO56MDOrkyLi3eCmugWbNRu',
    api_key_primary_val='1yiWJOyxczG1Ichq95JEUty5cZKr3F5iBynZfQWL',
    request_id='c157480f-7a49-4fff-bc49-65227afaaffb'
)

### **2-3. Hyperclova X에게 제공할 instruction과 example.**

- instruction : 뉴스 기사가 제공되었을 때 어떠한 형식으로 객체 및 관계의 정보를 추출해야하는지, Hyperclova X가 수행해야할 과제를 구체적으로 지시하는 지문.

- example : 과제 수행의 예시.

In [ ]:
instruction = {"role":"user","content":"다음의 과제를 수행해줘.\r\n\r\n-목표-\r\n뉴스 기사가 주어졌을 때, 제시된 객체 유형에 포함되는 객체들과 해당 객체 간의 관계를 모두 파악하기.\r\n\r\n-객체 유형-\r\n1. 인물: 실존하는 사람에 해당하는 객체.\r\n2. 기업: 실존하는 기업에 해당하는 객체.\r\n3. 국가: 실존하는 국가 또는 국가에서 운영하는 기관에 해당하는 객체.\r\n4. 기술: 사람이나 기업이 개발한 기술에 해당하는 객체.\r\n5. 사건: 인물이나 기업 객체들에 중대한 영향을 끼쳤을 만한 사건에 대한 객체. 단, 사건이 독립적으로 인물이나 기업에 영향을 준 경우에만 객체로 식별. \n5-1. 사건 객체인 예시:  \"코로나19로 인해 쿠팡의 매출이 감소했다.\"라는 문장에서는 \"코로나19\"라는 사건이 독랍적으로 \"쿠팡\"이라는 기업에 영향을 주었기에 추출해야함. \n5-2. 사건 객체가 아닌 예시: \"삼성과 애플의 특허분쟁\"은 \"삼성\"이라는 기업 객체와 \"애플\"이라는 기업 객체 간에 발생한  종속적인 사건이기 때문에 객체로 추출할 필요가 없음. \n\r\n-단계-\r\n1. 뉴스 기사에서 중요하다고 판단되는 객체들을 식별하기. 식별된 객체 별로 다음의 정보를 추출하기:\r\n- 객체_이름: 객체의 이름.\r\n- 객체_유형: 객체 유형 목록에서 식별된 객체와 가장 적합한 유형 하나를 선택.\r\n- 객체_설명: 객체의 특성과 활동에 대한 종합적인 설명.\r\n(객체[구분]<객체_이름>[구분]<객체_유형>[구분]<객체_설명>)의 형식으로 객체 정보를 구성.\r\n\r\n2. 1단계에서 얻은 객체들 중 서로 명확하게 연관되어있는 모든 객체 쌍 (객체_1, 객체_2)를 식별하기. \r\n각 연관된 객체 쌍 별로 다음의 정보를 추출하기:\r\n- 객체_1: 1단계에서 식별된 객체의 이름.\r\n- 객체_2: 1단계에서 식별된 객체의 이름.\r\n- 관계_설명: 객체_1과 객체_2가 서로 연관되어있다고 생각하는 이유에 대한 설명.\r\n- 관계_강도: 객체_1과 객체_2 사이의 관계가 각 객체와 연관된 기업의 주가 관점에서 긍정적인지, 또는 부정적인지를 나타내는 점수로 -5에서 +5의 범위를 가지며 양수는 객체_1과 객체_2의 주가(혹은 객체_1과 객체_2와 연관된 기업의 주가)에 있어서 긍정적인 관계, 음수는 부정적인 관계를 나타낸다. 관계 강도의 절대값이 클수록 영향력이 강한 관계를 의미한다.\r\n- 관계_날짜: 뉴스 기사 작성일.\r\n- 관계_출처: 뉴스 기사 URL.\r\n(관계[구분]<객체_1>[구분]<객체_2>[구분]<관계_설명>[구분]<관계_강도>[구분]<관계_날짜>[구분]<관계_출처>)의 형식으로 관계 정보를 구성.\r\n\r\n3. 1단계와 2단계에서 얻은 모든 객체와 관계의 정보를 출력하기."}
example = {"role":"user","content":"앞서 언급한 과제의 예시를 내가 너에게 제공할 거야. 이 예시를 참고해서 이후에 내가 요청할 과제를 수행해줘. 예시에 대한 과제는 수행하지 않아도 돼.\n\n예시:\r\n\r\n뉴스 기사 제목: 코로나 타격 온도차...면세점 최악, 백화점 심각, 쿠팡 호황\r\n\r\n뉴스 기사 작성일: 2020-03-22\r\n\r\n뉴스 기사 URL: https://n.news.naver.com/mnews/article/088/0000890336\r\n\r\n뉴스 기사 본문:\r\n\r\n인천공항 하루 이용객 수 20만 명→만 명 선으로 감소\r\n면세점업계 매출 80% 감소…백화점업계 30% 이상 감소\r\n대형마트들은 온라인 쇼핑 늘어 평소 매출 유지\r\n온라인 전문 쇼핑몰과 새벽 배송업체는 매출 증가\r\n[앵커]\r\n지난 1월 20일 국내 코로나19 첫 확진자가 나온 지 두 달이 지난 지금 유통업계는 매출 하락으로 크게 고전하고 있지만, 업종별로 희비가 엇갈리고 있습니다.\r\n면세점업계는 최악의 위기 상황이고 백화점업계도 심각하지만 소비자들이 비대면 쇼핑을 늘림에 따라 온라인 쇼핑업체들은 매출이 늘었습니다.\r\n박병한 기자가 보도합니다.\r\n[기]\r\n코로나19 사태로 국제선 항공편이 대부분 중단된 요즘 인천공항의 이용자 수가 평소 20만 명 선에서 만 명 선으로 줄었습니다.\r\n이에 따라 면세업계 매출은 지난해 같은 기간에 비해 80% 감소했고 김포공항 국제선 청사에 입점한 면세점들은 휴업에 들어갔습니다.\r\n백화점 업계도 코로나19 후유증을 톡톡히 겪고 있습니다.\r\n이달 1일부터 15일까지 주요 백화점 업계 매출은 지난해 같은 기간에 비해 30% 이상 줄었습니다.\r\n롯데백화점은 41.7%, 신세계백화점은 34.2%, 현대백화점은 32.1%의 매출 감소를 각각 기록했습니다.\r\n반면 대형마트들은 사람들이 매장 출입을 기피함에 따라 매장 매출은 다소 줄었지만 온라인 쇼핑 주문이 늘어 평소 수준의 매출을 유지하고 있는 것으로 알려졌습니다.\r\n온라인쇼핑몰 쿠팡은 코로나19 사태 이후 주문량이 30-40% 늘었고, 새벽배송 전문업체는 주문량이 5배가 느는 등 호황을 누리고 있습니다.\r\n전문가들은 코로나19 사태를 거치면서 소비자들의 쇼핑 행태도 변할 것으로 전망하고 있습니다.\r\n[서용구 / 숙명여대 경영전문대학원장 : 대면 서비스에서 비대면 서비스로 소매업의 정체성이 바뀔 정도로 엄청난 소비자 행동 변화가 벌어지고 있습니다]\r\n이 때문인지 롯데와 신세계 등 전통적인 유통 강자들도 온라인 쇼핑몰을 합쳐 운영하거나 택배회사 인수에 나서는 등 온라인 쇼핑 강화를 서두르고 있습니다.\r\n\r\n답변:\r\n\r\n(객체[구분]쿠팡[구분]기업[구분]쿠팡은 대한민국의 온라인 쇼핑몰로, 새벽배송 서비스와 다양한 상품을 제공하며 급성장 중이다.)\r\n(객체[구분]롯데쇼핑[구분]기업[구분]롯데쇼핑은 대한민국의 대형 유통 기업으로, 백화점, 마트 등 다양한 오프라인 및 온라인 쇼핑 플랫폼을 운영하고 있다.)\r\n(객체[구분]현대백화점[구분]기업[구분]현대백화점은 대한민국의 대형 백화점 체인으로, 다양한 패션, 가전, 식료품 등을 판매하고 있다.)\r\n(객체[구분]코로나19[구분]사건[구분]코로나19는 2019년 말 처음 발생한 신종 코로나바이러스 감염증으로, 전 세계적인 팬데믹을 일으켰다.)\r\n(관계[구분]코로나19[구분]쿠팡[구분]코로나19의 확산으로 인해 쿠팡의 주문량이 30-40% 증가하고, 새벽배송 주문량이 5배 증가하는 호황을 맞이하고 있다.[구분]+4[구분]2020-03-22[구분]https://n.news.naver.com/mnews/article/088/0000890336)\r\n(관계[구분]코로나19[구분]롯데쇼핑[구분]코로나19의 확산으로 인해 롯데쇼핑의 오프라인 매출이 감소하였으나, 온라인 쇼핑몰 통합 운영을 강화하고 있다.[구분]-3[구분]2020-03-22[구분]https://n.news.naver.com/mnews/article/088/0000890336)\r\n(관계[구분]코로나19[구분]현대백화점[구분]코로나19의 확산으로 인해 현대백화점의 매출이 30% 이상 감소하고 있다.[구분]-4[구분]2020-03-22[구분]https://n.news.naver.com/mnews/article/088/0000890336)\n"}

print(instruction["content"] + '\n')
print(example["content"] + '\n')

### **2-4. Hyperclova X를 활용한 객체 및 관계 추출.**

- data/news_info_example.json에 저장된 뉴스 정보를 Hyperclova X에 입력하여 객체 및 관계를 추출.

- results/extracted_result_example.txt에 추출 결과를 저장.

In [ ]:
with open("data/news_info_example.json", "r") as f:
    news_info = json.load(f)

fixed_messages = [instruction, example]
messages = []

for index in news_info.keys():
    news_title = "뉴스 기사 제목: " + news_info[index]['title'] + "\n\n"
    news_time = "뉴스 기사 작성일: " + news_info[index]['date'] + "\n\n"
    news_url = "뉴스 기사 작성일: " + news_info[index]['source'] + "\n\n"
    news_document = "뉴스 기사 본문: " + "\n\n" + news_info[index]['document'] 
        
    messages.append({"role": "user", "content": news_title + news_time + news_url + news_document})
    messages.append(instruction)
    messages.append(example)
    
    sliding_window_request_data = {
        "messages": messages,
        "maxTokens": 1000
    }

    try:
        adjusted_messages = sliding_window_executor.execute(sliding_window_request_data)
        if adjusted_messages == 'Error':
            print("Error adjusting messages with SlidingWindowExecutor")
            continue
    except Exception as e:
        print(f"Error adjusting messages: {e}")
        continue
    
    if len(adjusted_messages) <= 2:
        continue
    
    adjusted_messages.pop()
    adjusted_messages.pop()
    adjusted_messages = fixed_messages + adjusted_messages  

    completion_request_data = {
        "messages": adjusted_messages,
        "maxTokens": 1000, 
        "temperature": 0.5,
        "topK": 0,
        "topP": 0.8,
        "repeatPenalty": 1.2,
        "stopBefore": [],
        "includeAiFilters": True,
        "seed": 0
    }
    
    try:
        response = completion_executor.execute(completion_request_data)
        extract_result = response.get('result', {}).get('message', {}).get('content', '').strip()
        messages.append({"role": "assistant", "content": extract_result})
        print("NEWS {}:\n\n{}".format(index, news_title + news_time + news_url + news_document))
        print("\nCLOVA Response:\n\n{}\n".format(extract_result))
        with open("results/extracted_result_example.txt", 'a') as f:
            start = extract_result.find("(")
            extract_result = extract_result[start:].replace("\n\n", "\n")
            f.write(extract_result + "\n\n")
            
    except Exception as e:
        print(f"Error: {e}")